# Test OpenAI Agents SDK
- Implement a workflow to write a daily AI newsletter
- see README.md for details


In [1]:
import os
import yaml
import dotenv
import logging
import json
import yaml
from datetime import datetime
import time
import random
import glob
import pickle
import sqlite3

from pathlib import Path

import asyncio
import nest_asyncio

import pydantic
from pydantic import BaseModel, Field, RootModel
from typing import Dict, TypedDict, Type, List, Optional, Any, Iterable, Text
from dataclasses import dataclass, field
from enum import Enum

import numpy as np
import pandas as pd

import pandas as pd
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import hdbscan

import openai
from openai import AsyncOpenAI

import agents
from agents.exceptions import InputGuardrailTripwireTriggered
from agents import (Agent, Runner, Tool, OpenAIResponsesModel, 
                    ModelSettings, FunctionTool, InputGuardrail, GuardrailFunctionOutput,
                    SQLiteSession, set_default_openai_api, set_default_openai_client
                   )


import tenacity
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

from IPython.display import HTML, Image, Markdown, display

from log_handler import SQLiteLogHandler, setup_sqlite_logging, sanitize_error_for_logging
from config import LOGDB
from llm import LLMagent, LangfuseClient  # methods to apply prompts async to large batches
from db import Url 

from fetch import Fetcher # fetch news urls
from newsletter_state import NewsletterAgentState, StepStatus
from news_agent import NewsletterAgent


In [2]:
print(f"OpenAI:            {openai.__version__}")
print(f"OpenAI Agents SDK  {agents.__version__}")
print(f"Pydantic           {pydantic.__version__}")


OpenAI:            1.109.0
OpenAI Agents SDK  0.3.1
Pydantic           2.11.9


In [3]:
dotenv.load_dotenv()

# to run async in jupyter notebook
nest_asyncio.apply()

# verbose OpenAI console logging if something doesn't work
# logging.basicConfig(level=logging.DEBUG)
# openai_logger = logging.getLogger("openai")
# openai_logger.setLevel(logging.DEBUG)


In [4]:
# modules create a default logger, or we can pass this logger

def setup_logging(session_id: str = "default", db_path: str = "agent_logs.db") -> logging.Logger:
    """Set up logging to console and SQLite database."""

    # Create logger
    logging.basicConfig(level=logging.INFO)

    logger = logging.getLogger(f"NewsletterAgent.{session_id}")
    logger.setLevel(logging.INFO)

    # Clear any existing handlers
    logger.handlers.clear()

    # Console handler
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    console_formatter = logging.Formatter(
        '%(asctime)s | %(name)s | %(levelname)s | %(message)s',
        datefmt='%H:%M:%S'
    )
    console_handler.setFormatter(console_formatter)

    # SQLite handler
    sqlite_handler = SQLiteLogHandler(db_path)
    sqlite_handler.setLevel(logging.INFO)
    sqlite_formatter = logging.Formatter('%(message)s')
    sqlite_handler.setFormatter(sqlite_formatter)

    # Add handlers to logger
    logger.addHandler(console_handler)
    logger.addHandler(sqlite_handler)

    # Prevent propagation to root logger
    logger.propagate = False

    return logger

logger = setup_logging("newsletter_agent", "test_logs.db")

# Log some test messages
logger.info("Test info message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.warning("Test warning message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.error("Test error message", extra={
    'step_name': 'error_step',
    'agent_session': 'demo_session'
})

sanitize_error_for_logging("log with some bad stuff for the filter: sk-proj-123456789012345678901234567890123456789012345678")

21:12:31 | NewsletterAgent.newsletter_agent | INFO | Test info message
21:12:31 | NewsletterAgent.newsletter_agent | WARNING | Test warning message
21:12:31 | NewsletterAgent.newsletter_agent | ERROR | Test error message


'log with some bad stuff for the filter: [API_KEY_REDACTED]'

# Run Agent Worfklow

In [5]:
print("🚀 Creating NewsletterAgent...")

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set")

# Set up OpenAI client for the agents SDK
set_default_openai_client(AsyncOpenAI(api_key=api_key))
try:
    # set up state
    session_id = 'test_newsletter_20250928201328725764'
    step_name = 'step_05_cluster_by_topic'
#     del session_id
except Exception as e:
    print(e)

do_download=False
process_since=None
process_since='2025-09-28 11:00:00'

# Create agent with persistent state
if 'session_id' in vars():
    # load state from db for session_id and state
    print("session_id is defined")
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db", 
                                 do_download=do_download,
                                 process_since=process_since,
                                 verbose=True
                                )
    state = state.load_from_db(step_name)
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=True, timeout=30)    
else:
    # create new session
    print("session_id is not defined")
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")    
    session_id = f"test_newsletter_{timestamp}"
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db",
                                 do_download=do_download,
                                 process_since=process_since,
                                 verbose=True
                                ) 
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=False, timeout=30)
    state.serialize_to_db("initialize")

21:12:35 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Using provided state with 168 articles


🚀 Creating NewsletterAgent...
session_id is defined
test_newsletter_20250928201328725764
Initialized NewsletterAgent with persistent state and 9-step workflow
Session ID: test_newsletter_20250928201328725764


In [8]:
agent.state.get_status()


{'headlines': {'total': 168},
 'sources': {'config_file': 'sources.yaml', 'loaded_sources': 0},
 'topics': {'cluster_topics': 0, 'topics': []},
 'workflow': {'current_step': 'step_06_rate_articles',
  'workflow_complete': False,
  'workflow_status': 'started',
  'workflow_status_message': '',
  'progress_percentage': 55.55555555555556,
  'max_edits': 2,
  'concurrency': 16},
 'processing': {'topic_clusters': 0,
  'newsletter_sections': 0,
  'final_newsletter_length': 0}}

In [16]:
state.get_current_step()


'step_01_fetch_urls'

In [17]:
# User prompt to run workflow
user_prompt = "Show the workflow status"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


📝 User prompt: 'Show the workflow status'


20:13:34 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Starting check_workflow_status
20:13:34 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Completed check_workflow_status


⏱️  Total execution time: 5.68s
📊 Final result:
Current workflow status:
- Progress: 0.0% (0/9 complete)
- Status summary: 0 complete, 0 started, 0 failed, 9 not started
- Next step: Step 1 — Fetch URLs

Step details:
- Step 1: Fetch Urls: not_started
- Step 2: Filter Urls: not_started
- Step 3: Download Articles: not_started
- Step 4: Extract Summaries: not_started
- Step 5: Cluster By Topic: not_started
- Step 6: Rate Articles: not_started
- Step 7: Select Sections: not_started
- Step 8: Draft Sections: not_started
- Step 9: Finalize Newsletter: not_started

Would you like me to start Step 1 (gather URLs) and continue through the workflow, or run a specific step?


In [18]:
# User prompt to run a workflow step
user_prompt = "Run step 1, fetch urls"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)



📝 User prompt: 'Run step 1, fetch urls'


20:13:42 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Starting check_workflow_status
20:13:42 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Completed check_workflow_status
20:13:43 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Starting Step 1: Gather URLs
2025-09-28 20:13:43,903 - fetcher_5084558864 - INFO - [fetcher_init] Loading sources from sources.yaml
2025-09-28 20:13:43,919 - fetcher_5084558864 - INFO - [fetcher_init] Loaded 17 sources: 7 RSS, 9 HTML, 1 API
2025-09-28 20:13:43,919 - fetcher_5084558864 - DEBUG - [fetcher_sources] Source 'Ars Technica': type=RSS, url=https://arstechnica.com/ai/
2025-09-28 20:13:43,920 - fetcher_5084558864 - DEBUG - [fetcher_sources] Source 'Bloomberg': type=RSS, url=https://www.bloomberg.com/ai
2025-09-28 20:13:43,920 - fetcher_5084558864 - DEBUG - [fetcher_sources] Source 'Business Insider': type=html, url=https://www.businessinsider.com/tech
2025-09-28 20:13:43,920 - fetcher_5084558864 - DEBU

▶ Starting Step 1: step_01_fetch_urls


2025-09-28 20:13:44,185 - fetcher_5084558864 - INFO - [fetch_html] Parsed HTML file: download/sources/Feedly_AI.html
2025-09-28 20:13:44,185 - fetcher_5084558864 - INFO - [fetch_html] HTML fetch successful for Feedly AI: 105 articles
2025-09-28 20:13:44,186 - fetcher_5084558864 - INFO - [fetch_rss] Fetching RSS from Hacker News: https://news.ycombinator.com/rss
2025-09-28 20:13:44,186 - fetcher_5084558864 - INFO - [fetch_rss] Fetching RSS from HackerNoon: https://hackernoon.com/tagged/ai/feed
2025-09-28 20:13:44,188 - fetcher_5084558864 - INFO - [fetch_rss] Fetching RSS from New York Times: https://rss.nytimes.com/services/xml/rss/nyt/Technology.xml
2025-09-28 20:13:44,189 - fetcher_5084558864 - INFO - [fetch_html] Using existing HTML file from Reddit: https://www.reddit.com/r/AI_Agents+ArtificialInteligence+Automate+ChatGPT+ChatGPTCoding+Futurology+MachineLearning+OpenAI+ProgrammerHumor+accelerate+aiArt+aivideo+artificial+deeplearning+learnmachinelearning+programming+singularity+tech+

,source,url
0,Ars Technica,20
1,Bloomberg,27
2,Business Insider,15
3,FT,50
4,Feedly AI,102
5,Hacker News,30
6,HackerNoon,50
7,New York Times,23
8,NewsAPI,28
9,Reddit,49


,source,title,url,published,rss_summary,id
0,Ars Technica,Why LA Comic Con thought making an AI-powered ...,https://arstechnica.com/ai/2025/09/why-la-comi...,"Sat, 27 Sep 2025 11:00:07 +0000",“I suppose if we do it and thousands of fans… ...,0
1,Ars Technica,Can AI detect hedgehogs from space? Maybe if y...,https://arstechnica.com/ai/2025/09/can-ai-dete...,"Fri, 26 Sep 2025 22:22:13 +0000",Cambridge researchers use satellite-based bram...,1
2,Ars Technica,YouTube Music is testing AI hosts that will in...,https://arstechnica.com/google/2025/09/youtube...,"Fri, 26 Sep 2025 21:05:30 +0000",YouTube Labs will be a place to preview all th...,2
3,Ars Technica,ChatGPT Pulse delivers morning updates based o...,https://arstechnica.com/ai/2025/09/chatgpt-pul...,"Thu, 25 Sep 2025 20:30:52 +0000",New mobile chatbot feature analyzes conversati...,3
4,Ars Technica,Experts urge caution about using ChatGPT to pi...,https://arstechnica.com/information-technology...,"Thu, 25 Sep 2025 18:10:50 +0000",AI-selected portfolios might perform well in a...,4
...,...,...,...,...,...,...
556,NewsAPI,Libra split? More clubs condemn Flamengo’s mov...,https://onefootball.com/en/news/libra-split-mo...,2025-09-27T22:27:00Z,NaN,556
557,NewsAPI,Why Advertisers Are Returning to Big Oil Despi...,https://oilprice.com/Energy/Energy-General/Why...,2025-09-27T23:00:00Z,NaN,557
558,NewsAPI,Show HN: AI Video Generator app for iOS (looki...,https://apps.apple.com/us/app/ai-video-generat...,2025-09-27T23:05:00Z,NaN,558
559,NewsAPI,Overcoming all odds for Oxford,https://www.thestar.com.my/news/education/2025...,2025-09-27T23:00:00Z,NaN,559


20:13:44 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Completed Step 1: Gathered 646 articles
20:13:46 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Starting check_workflow_status
20:13:46 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Completed check_workflow_status


⏱️  Total execution time: 9.46s
📊 Final result:
Step 1 completed.

Summary:
- Fetched URLs and headlines from sources (RSS): 646 collected, 561 stored in persistent state.
- Workflow progress: 11.1% (1/9 complete)
- Next step: Step 2 — Filter URLs (to AI-related content)

Would you like me to continue and run Step 2 (filter URLs) now?


In [19]:
pd.DataFrame(state.headline_data) 


,source,title,url,published,rss_summary,id
0,Ars Technica,Why LA Comic Con thought making an AI-powered ...,https://arstechnica.com/ai/2025/09/why-la-comi...,"Sat, 27 Sep 2025 11:00:07 +0000",“I suppose if we do it and thousands of fans… ...,0
1,Ars Technica,Can AI detect hedgehogs from space? Maybe if y...,https://arstechnica.com/ai/2025/09/can-ai-dete...,"Fri, 26 Sep 2025 22:22:13 +0000",Cambridge researchers use satellite-based bram...,1
2,Ars Technica,YouTube Music is testing AI hosts that will in...,https://arstechnica.com/google/2025/09/youtube...,"Fri, 26 Sep 2025 21:05:30 +0000",YouTube Labs will be a place to preview all th...,2
3,Ars Technica,ChatGPT Pulse delivers morning updates based o...,https://arstechnica.com/ai/2025/09/chatgpt-pul...,"Thu, 25 Sep 2025 20:30:52 +0000",New mobile chatbot feature analyzes conversati...,3
4,Ars Technica,Experts urge caution about using ChatGPT to pi...,https://arstechnica.com/information-technology...,"Thu, 25 Sep 2025 18:10:50 +0000",AI-selected portfolios might perform well in a...,4
...,...,...,...,...,...,...
556,NewsAPI,Libra split? More clubs condemn Flamengo’s mov...,https://onefootball.com/en/news/libra-split-mo...,2025-09-27T22:27:00Z,NaN,556
557,NewsAPI,Why Advertisers Are Returning to Big Oil Despi...,https://oilprice.com/Energy/Energy-General/Why...,2025-09-27T23:00:00Z,NaN,557
558,NewsAPI,Show HN: AI Video Generator app for iOS (looki...,https://apps.apple.com/us/app/ai-video-generat...,2025-09-27T23:05:00Z,NaN,558
559,NewsAPI,Overcoming all odds for Oxford,https://www.thestar.com.my/news/education/2025...,2025-09-27T23:00:00Z,NaN,559


In [20]:
countdf = pd.DataFrame(state.headline_data) \
    .groupby("source") \
    .count()[["id"]] \
    .reset_index() \
    .rename(columns={'id': 'count'}) \
    .sort_values("count", ascending=False)
countdf 


,source,count
4,Feedly AI,102
12,The Register,50
3,FT,50
6,HackerNoon,50
9,Reddit,49
5,Hacker News,30
8,NewsAPI,28
16,Washington Post,28
1,Bloomberg,27
7,New York Times,23


In [21]:
# Run tool directly without LLM processing an input prompt or results
# user_prompt = "Run step 2, filter urls"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("filter_urls")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


20:13:48 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Starting Step 2: Filter URLs
20:13:48 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | 🔍 Filtering 561 headlines...
20:13:48 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | 🔄 Checking for duplicates seen before 2025-09-28T11:00:00
20:13:48 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | 🔍 Filtering 561 articles for dupes.
20:13:48 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | URL deduplication with process_since: 234 URLs filtered (seen before 2025-09-28T11:00:00), 327 new URLs remain
20:13:48 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | 🔍 Filtering 561 headlines for AI relevance using LLM...
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/filter_urls' from Langfuse
INFO:llm:Parsed prompt 'newsagent/filter_urls': model=gpt-4.1-mini, system_len=459, user_len=954


▶ Starting Step 2: step_02_filter_urls
checking https://arstechnica.com/ai/2025/09/why-la-comic-con-thought-making-an-ai-powered-stan-lee-hologram-was-a-good-idea/
found before cutoff
checking https://arstechnica.com/ai/2025/09/can-ai-detect-hedgehogs-from-space-maybe-if-you-find-brambles-first/
found before cutoff
checking https://arstechnica.com/google/2025/09/youtube-music-is-testing-ai-hosts-that-will-interrupt-your-tunes/
found before cutoff
checking https://arstechnica.com/ai/2025/09/chatgpt-pulse-delivers-morning-updates-based-on-your-chat-history/
found before cutoff
checking https://arstechnica.com/information-technology/2025/09/experts-urge-caution-about-using-chatgpt-to-pick-stocks/
found before cutoff
checking https://arstechnica.com/google/2025/09/google-deepmind-unveils-its-first-thinking-robotics-ai/
found before cutoff
checking https://arstechnica.com/science/2025/09/deepminds-robotic-ballet-an-ai-for-coordinating-manufacturing-robots/
found before cutoff
checking https

20:14:04 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Completed Step 2: 172 AI-related articles, 234 duplicates removed


⏱️  Total execution time: 16.86s
📊 Final result:
✅ Step 2 step_02_filter_urls completed successfully! Removed 234 duplicate URLs, classified 327 new articles, found 172 AI-related.


In [22]:
# User prompt to run workflow
# user_prompt = "Run step 3, download full articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("download_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

20:14:19 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Starting Step 3: Download Articles
20:14:19 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Starting concurrent scraping of 172 AI-related articles


▶ Starting Step 3: step_03_download_articles


20:14:19 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Launching browser for 172 URLs with 16 concurrent workers
20:14:21 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Worker 0 fetching 1 of 172 https://www.yahoo.com/news/articles/uae-president-meets-openai-ceo-220423052.html
20:14:21 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | scrape_url(https://www.yahoo.com/news/articles/uae-president-meets-openai-ceo-220423052.html)
20:14:21 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | scraping https://www.yahoo.com/news/articles/uae-president-meets-openai-ceo-220423052.html to download/html
20:14:21 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Downloading https://www.yahoo.com/news/articles/uae-president-meets-openai-ceo-220423052.html
20:14:21 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Worker 1 fetching 2 of 172 https://www.liquid.ai/press/liquid-unveils-nanos-extremely-small-foundat

20:14:21 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Worker 6 completed https://www.wsj.com/tech/ai/ai-bubble-building-spree-55ee6128 with status: success
20:14:21 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Worker 6 fetching 10 of 172 https://www.theverge.com/news/787042/trump-posts-then-pulls-bizarre-ai-video-promoting-medbed-conspiracy
20:14:21 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | scrape_url(https://www.theverge.com/news/787042/trump-posts-then-pulls-bizarre-ai-video-promoting-medbed-conspiracy)
20:14:21 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | scraping https://www.theverge.com/news/787042/trump-posts-then-pulls-bizarre-ai-video-promoting-medbed-conspiracy to download/html
20:14:21 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Downloading https://www.theverge.com/news/787042/trump-posts-then-pulls-bizarre-ai-video-promoting-medbed-conspiracy
20:14:21 | NewsletterAgent.test_new

20:14:23 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | scrape_url(https://natlawreview.com/article/all-china-patent-attorneys-association-bans-members-using-ai-generate-patent)
20:14:23 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | scraping https://natlawreview.com/article/all-china-patent-attorneys-association-bans-members-using-ai-generate-patent to download/html
20:14:23 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Downloading https://natlawreview.com/article/all-china-patent-attorneys-association-bans-members-using-ai-generate-patent
20:14:23 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Worker 11 fetching 20 of 172 http://www.techmeme.com/250928/p14#a250928p14
20:14:23 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | scrape_url(http://www.techmeme.com/250928/p14#a250928p14)
20:14:23 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | scraping http://www.techmeme.com/250928/p14#a2509

...

20:48:30 | NewsletterAgent.test_newsletter_20250928201328725764 | ERROR | Unexpected error scraping https://www.ft.com/content/c610a0a0-99b2-47eb-9536-e948c80a550e: Mouse.move: Target page, context or browser has been closed
Browser logs:

[pid=66632][err] JavaScript error: , line 0: TypeError: NetworkError when attempting to fetch resource.
[pid=66632][err] JavaScript error: , line 0: TypeError: NetworkError when attempting to fetch resource.
[pid=66632][err] JavaScript error: , line 0: TypeError: NetworkError when attempting to fetch resource.
[pid=66632][err] JavaScript error: , line 0: TypeError: NetworkError when attempting to fetch resource.
[pid=66632][err] JavaScript error: , line 0: TypeError: NetworkError when attempting to fetch resource.
[pid=66632][err] JavaScript error: , line 0: TypeError: NetworkError when attempting to fetch resource.
[pid=66632][err] JavaScript error: , line 0: TypeError: NetworkError when attempting to fetch resource.
[pid=66632][err] JavaScript erro

20:48:30 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Worker 12 completed http://www.techmeme.com/250928/p13#a250928p13 with status: success
20:48:30 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Worker 6 completed https://www.thewrap.com/donald-trump-posts-deletes-ai-generated-fox-news-medbeds/ with status: success
20:48:30 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Worker 3 completed https://www.rcrwireless.com/20250928/chipsets/extension-voice-ai-era with status: success
20:48:30 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Worker 5 completed https://news.google.com/rss/articles/CBMimgFBVV95cUxNMHF2eHRDUGhBOFpmSi1URFNGZExMZjNoNGJSTXNDX2ZMLW5uN29nc3Q0RVZkQndWaS11VjJXeFcyeUtXNEU2dzBCbDJkLXQwalo4aEVXMGtsRjgxTlg0bHRHR09wTmR0aC01aFdTZDgxaWV1dGRPTGhpQ2lJeTJBOTdHY1AzellvUXdFaWdPY0ZuWDF1SDBwWXZn with status: success
20:48:30 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Worker 9 completed https://

Starting with 172 rows...
Processing 172 files...
Reading and truncating files to 8192 tokens using text-embedding-3-large tokenizer...
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2

20:49:13 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Completed Step 3: Downloaded 134 articles


Creating indexed similarity matrix...
Finding pairs with similarity > 0.925...
Filtering dataframe...
  Pair: 132 vs 77
    132: Techmeme - Since 2019, Brazil's courts have developed or implemented over 140 AI projects that have helped make the country's overburdened judicial system more efficient (Pedro Nakamura/Rest of World)
    77: Feedly AI - Since 2019, Brazil's courts have developed or implemented over 140 AI projects that have helped make the country's overburdened judicial system more efficient (Pedro Nakamura/Rest of World)
  Pair: 123 vs 47
    123: Reddit - Everyone's wondering if, and when, the AI bubble will pop. Here's what went down 25 years ago that ultimately burst the dot-com boom | Fortune
    47: Feedly AI - Everyone's wondering if, and when, the AI bubble will pop. Here's what went down 25 years ago that ultimately burst the dot-com boom
  Pair: 137 vs 116
    137: Techmeme - Education software company EdSights, which uses SMS texting and AI to reach students and 

In [23]:
# User prompt to run workflow
# user_prompt = "Run step 4, Summarize articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("extract_summaries")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

20:49:13 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Processing 168 AI articles for summarization
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/extract_summaries' from Langfuse
INFO:llm:Parsed prompt 'newsagent/extract_summaries': model=gpt-4.1-mini, system_len=1204, user_len=43
20:49:13 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Using model 'gpt-4.1-mini' for summarization
20:49:13 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Starting summarization for 168 articles


▶ Starting Step 4: step_04_extract_summaries


20:49:19 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Extracting metadata from HTML files for 168 articles


⏱️  Total execution time: 9.81s
📊 Final result:
✅ Step 4 step_04_extract_summaries completed successfully! Generated AI-powered summaries for 168/168 articles.
💾 Summaries stored in headline DataFrame.


In [24]:
headline_df = state.headline_df
headline_df.loc[headline_df["url"]!= headline_df["final_url"]]

,source,title,url,published,rss_summary,id,isAI,status,final_url,html_path,last_updated,text_path,content_length,domain,site_name,reputation,summary,description,tags
13,Feedly AI,5 big analyst AI moves: Nvidia target upped to...,https://news.google.com/rss/articles/CBMizAFBV...,NaN,NaN,208,True,success,https://www.investing.com/news/stock-market-ne...,download/html/5_big_analyst_AI_moves__Nvidia_t...,2025-09-28T08:00:08Z,download/text/5_big_analyst_AI_moves__Nvidia_t...,10575,investing.com,Investing.com,0.0,- Barclays raised Nvidia's price target to $24...,5 big analyst AI moves: Nvidia target upped to...,[Stock Markets]
14,Feedly AI,Building smarter apps: Why machine learning se...,https://news.google.com/rss/articles/CBMiqwFBV...,NaN,NaN,124,True,success,https://azbigmedia.com/business/building-smart...,download/html/Building_smarter_apps__Why_machi...,2025-09-27T00:00:00Z,download/text/Building_smarter_apps__Why_machi...,10470,azbigmedia.com,AZ Big Media,0.0,- Machine learning is now essential across ind...,Building smarter apps: Here's why machine lear...,[]
21,Feedly AI,7 AI-Proof Jobs For Introverts Who Want Stability,https://news.google.com/rss/articles/CBMirgFBV...,NaN,NaN,138,True,success,https://www.forbes.com/sites/carolinecastrillo...,download/html/7_AI-Proof_Jobs_For_Introverts_W...,2025-09-28T20:30:00Z,download/text/7_AI-Proof_Jobs_For_Introverts_W...,9253,forbes.com,Forbes,1.0,- The article identifies seven AI-proof jobs w...,AI-proof jobs for introverts do exist. Explore...,"[Careers, artificial intelligence, AI, future ..."
29,Feedly AI,Where The AI Action Plan Falls Short On Health...,https://news.google.com/rss/articles/CBMikwFBV...,NaN,NaN,164,True,success,https://medcitynews.com/2025/09/where-the-ai-a...,download/html/Where_The_AI_Action_Plan_Falls_S...,2025-09-28T13:50:00Z,download/text/Where_The_AI_Action_Plan_Falls_S...,7570,medcitynews.com,MedCity News,0.0,- The U.S. AI Action Plan aims to build trust ...,"There are aspects of the plan worth praising, ...","[MedCity Influencers, Artificial Intelligence,..."
43,Feedly AI,"With AI Citations, Each Reference Is A Rabbit ...",https://news.google.com/rss/articles/CBMitAFBV...,NaN,NaN,176,True,success,https://www.forbes.com/sites/rheawessel/2025/0...,download/html/With_AI_Citations__Each_Referenc...,2025-09-28T16:12:53Z,download/text/With_AI_Citations__Each_Referenc...,6530,forbes.com,Forbes,1.0,- AI-generated content and citations often inc...,AI citations now mean sources of unknown origi...,[Careers]
59,Feedly AI,SuperQ Quantum Releases Post-Quantum Cryptogra...,https://news.google.com/rss/articles/CBMinwFBV...,NaN,NaN,196,True,success,https://thequantuminsider.com/2025/09/28/super...,download/html/SuperQ_Quantum_Releases_Post-Qua...,2025-09-28T10:18:00Z,download/text/SuperQ_Quantum_Releases_Post-Qua...,4898,thequantuminsider.com,The Quantum Insider,0.0,"- SuperQ Quantum released Super™ PQC Analyst, ...",SuperQ Quantum has launched Super™ PQC Analyst...,[]
61,Feedly AI,Canada Joins The Global Push For Sovereign AI ...,https://news.google.com/rss/articles/CBMivgFBV...,NaN,NaN,194,True,success,https://www.forbes.com/sites/ronschmelzer/2025...,download/html/Canada_Joins_The_Global_Push_For...,2025-09-28T14:40:10Z,download/text/Canada_Joins_The_Global_Push_For...,4823,forbes.com,Forbes,1.0,- TELUS has launched Canada's first fully sove...,TELUS launched Canada’s first fully sovereign ...,"[artificial intelligence, AI, Canada, model, s..."
66,Feedly AI,Defense tech firm Kela buys AI startup to bols...,https://news.google.com/rss/articles/CBMisgFBV...,NaN,NaN,172,True,success,https://www.timesofisrael.com/defense-tech-fir...,download/html/Defense_tech_firm_Kela_buys_AI_s...,2025-09-29T10:04:05Z,download/text/Defense_tech_firm_Kela_buys_AI_s...,4142,timesofisrael.com,Times of Israel,0.0,- Israeli defense tech company Kela Technologi...,"In its first acquisition, Kela buys Pelanor to...",[]
69,HackerNoon,AI Startup Turns Open Source Code Reviews Into...,https://ha

In [25]:
# User prompt to run workflow
# user_prompt = "Run step 5, Cluster articles by topic"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("cluster_by_topic")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


20:49:53 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Starting topic extraction for clustering
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/extract_topics' from Langfuse
INFO:llm:Parsed prompt 'newsagent/extract_topics': model=gpt-4.1-mini, system_len=1100, user_len=80
20:49:53 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Using model 'gpt-4.1-mini' for topic extraction
20:49:53 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Processing 168 articles for topic extraction


▶ Starting Step 5: step_05_cluster_by_topic


20:50:00 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Successfully extracted 557 total topics across articles
20:50:00 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Starting canonical topic classification for 101 topics
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/can

INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized Langfu

INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_top

...

/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 

=== HDBSCAN Parameters ===
min_cluster_size:   6
min_samples:        3
n_components:       61
=== Clustering Quality Metrics ===
Number of clusters: 0
Noise points: 168 (100.0%)
=== Quality Scores ===

[I 2025-09-28 20:51:01,631] Trial 196 finished with value: 1.0 and parameters: {'n_components': 61, 'min_cluster_size': 6, 'min_samples': 3}. Best is trial 93 with value: -0.2073528429500118.
=== HDBSCAN Parameters ===
min_cluster_size:   5
min_samples:        3
n_components:       35
=== Clustering Quality Metrics ===
Number of clusters: 2
Noise points: 23 (13.7%)
Average cluster size: 72.5 ± 67.5
Cluster size range: 5 - 140
=== Quality Scores ===
Silhouette Score: 0.166 (higher is better)
Calinski-Harabasz Score: 8.0 (higher is better)
Davies-Bouldin Score: 1.038 (lower is better)
HDBSCAN Validity Index: 0.149
Composite Score: 0.157 (higher is better)

[I 2025-09-28 20:51:01,659] Trial 197 finished with value: -0.15706396272455958 and parameters: {'n_components': 35, 'min_cluster_size'

20:51:02 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | 1: AI's Impact on Economy, Jobs, and Society
20:51:02 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Artificial Intelligencein Stroke Care: A Narrative Review of Diagnostic, Predictive, and Workflow Applications (Stroke Care, Diagnostic Imaging, Workflow AI, AI Validation, Bias And Fairness, Ethical AI, Policy And Regulation)
One Law Sets South Korea's AI Policy—and One Weak Link Could Break It (AI Regulation, National Strategy, Governance, Policy And Regulation, Industrial Policy, Safety And Alignment, Ethics)
Walmart CEO Issues Wake-Up Call: 'AI Is Going to Change Literally Every Job' (Job Automation, Retail Workforce, AI Impact, Chatbots, Customer Service, Jobs And Careers, Job Training)
The (economic) AI apocalypse is nigh (Economic Bubble, Investment Risks, AI Industry, Finance, Job Automation, Policy Recommendations, Hardware)
Will AI Mean Bring an End to Top Programming Language Rankings? (

20:51:02 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | 

20:51:03 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | 0: Trump Shares AI-Generated MedBed Video
20:51:03 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Trump posts, then pulls bizarre AI video promoting MedBed conspiracy (Donald Trump, AI Misinformation, Deepfakes, Conspiracy Theories, Policy And Regulation, Safety And Alignment, Disinformation)
Trump Deletes Wild AI Video He Shared In Which He Promoted a Magic 'Med Bed' That Cures Diseases and Regrows Limbs (Donald Trump, Misinformation, Deepfakes, Medical Claims, Healthcare, Politics, Society And Culture)
The President posted an AI generated Fox News report announcing MedBeds, a technology that doesn’t exist (Privacy, Privacy And Surveillance, Cookies, Advertising, Policy And Regulation, Community Guidelines, Streaming)
Trump shares apparent AI video promoting ‘medbed’ conspiracy theory ()
Trump Posts and Deletes AI-Generate

⏱️  Total execution time: 70.82s
📊 Final result:
✅ Step 5 step_05_cluster_by_topic completed successfully! Organized 168 articles into topic clusters.


In [6]:
# User prompt to run workflow
# user_prompt = "Run step 6, Rate articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("rate_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

21:13:01 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Rating 168 AI articles using fn_rate_articles
21:13:01 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Calculating article ratings for 168 articles
21:13:01 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Rating recency
21:13:02 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Rating spam probability
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/rate_quality' from Langfuse
INFO:llm:Parsed prompt 'newsagent/rate_quality': model=gpt-4.1, system_len=1849, user_len=246


▶ Starting Step 6: step_06_rate_articles


21:13:07 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | low quality articles: {0.0: 112, 1.0: 29, 0.9999998063873693: 3, 0.2942149721551204: 2, 0.9999996871837232: 2, 0.8519527349543199: 1, 0.998073226003552: 1, 0.9999723899264568: 1, 0.29421493708196467: 1, 0.8175743455931945: 1, 0.9669140281575853: 1, 1.2790354113010132e-13: 1, 0.9997694306355145: 1, 2.172439935079017e-10: 1, 5.3157852544244216e-08: 1, 0.5621765025686553: 1, 0.3208212708736474: 1, 6.224144622907783e-11: 1, 3.059023205018258e-07: 1, 4.363462252943702e-09: 1, 4.152420021175756e-14: 1, 1.5581841173700477e-12: 1, 3.4663274126619643e-07: 1, 0.24508502137530588: 1, 0.9984987679933719: 1}
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/rate_on_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/rate_on_topic': model=gpt-4.1, system_len=1790, user_len=240
21:13:10 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | on topic articles: {1.0: 109, 0.0: 6, 0.9

✅ Completed Step 6: Rated 166 articles
⏱️  Total execution time: 247.55s
📊 Final result:
✅ Step 6 step_06_rate_articles completed successfully! Rated 166 articles with average rating 3.7/10.
⭐ High quality articles (≥7.0): 1
💾 Ratings stored in persistent state.


In [7]:
# User prompt to run workflow
user_prompt = "Show the workflow status"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


📝 User prompt: 'Show the workflow status'


21:19:52 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Starting check_workflow_status
21:19:52 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Completed check_workflow_status


⏱️  Total execution time: 5.85s
📊 Final result:
Current workflow status:
- Progress: 66.7% (6/9 complete)
- Status summary: 6 complete, 0 started, 0 failed, 3 not started
- Next step: Step 7 — Select Sections

Step details:
- Step 1: Fetch Urls: complete
- Step 2: Filter Urls: complete
- Step 3: Download Articles: complete
- Step 4: Extract Summaries: complete
- Step 5: Cluster By Topic: complete
- Step 6: Rate Articles: complete
- Step 7: Select Sections: not_started
- Step 8: Draft Sections: not_started
- Step 9: Finalize Newsletter: not_started

Data summary:
- Total articles: 166
- AI-related: 166

Would you like me to run Step 7 (organize articles into newsletter sections) now?


In [18]:
headline_df = state.headline_df.sort_values("bt_z", ascending=False)
headline_df    

,source,title,url,published,rss_summary,id,isAI,status,final_url,html_path,...,input_text,age,recency_score,low_quality,on_topic,important,bt_z,adjusted_len,rating,bradley_terry
134,VentureBeat,six different types of AI models called Liquid...,https://www.liquid.ai/press/liquid-unveils-nan...,None,None,134,True,success,https://www.liquid.ai/press/liquid-unveils-nan...,download/html/six_different_types_of_AI_models...,...,six different types of AI models called Liquid...,2.131516,-5.435642e-01,0.000000e+00,1.000000e+00,1.000000e+00,2.441206,0.945321,2.401757,26.295867
67,NewsAPI,The (economic) AI apocalypse is nigh,https://pluralistic.net/2025/09/27/econopocaly...,2025-09-27T22:30:00Z,None,67,True,success,https://pluralistic.net/2025/09/27/econopocaly...,download/html/The__economic__AI_apocalypse_is_...,...,The (economic) AI apocalypse is nigh\n- The ar...,0.000000,1.000000e+00,5.315785e-08,1.000000e+00,8.807970e-01,2.324497,1.259403,4.140200,25.038717
114,VentureBeat,"The $100M OpenAI partnership is nice, but Data...",https://venturebeat.com/ai/the-usd100m-openai-...,None,None,114,True,success,https://venturebeat.com/ai/the-usd100m-openai-...,download/html/The__100M_OpenAI_partnership_is_...,...,"The $100M OpenAI partnership is nice, but Data...",3.175718,-7.786682e-01,0.000000e+00,1.000000e+00,1.000000e+00,2.043228,0.816639,3.037971,22.008979
162,Feedly AI,Trump Posts and Deletes AI-Generated Fox News ...,https://www.thewrap.com/donald-trump-posts-del...,None,None,162,True,success,https://www.thewrap.com/donald-trump-posts-del...,,...,Trump Posts and Deletes AI-Generated Fox News ...,1.000000,-1.323719e-08,8.519527e-01,3.208213e-01,2.365978e-14,1.920206,0.000000,-0.531131,20.683830
80,Feedly AI,Researchers (Including Google) are Betting on ...,https://slashdot.org/story/25/09/27/0632215/re...,None,None,80,True,success,https://slashdot.org/story/25/09/27/0632215/re...,download/html/Researchers__Including_Google__a...,...,Researchers (Including Google) are Betting on ...,0.000000,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.722355,0.840859,3.840859,18.552638
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,Feedly AI,Wiz chief technologist Ami Luttwak on how AI i...,https://finance.yahoo.com/news/wiz-chief-techn...,None,None,146,True,success,https://finance.yahoo.com/news/wiz-chief-techn...,,...,Wiz chief technologist Ami Luttwak on how AI i...,1.000000,-1.323719e-08,1.000000e+00,9.626730e-01,0.000000e+00,-2.043171,0.000000,1.962673,-22.008367
26,Bloomberg,AI Boom Powers Triple-Digit Gains in China’s H...,https://www.bloomberg.com/news/articles/2025-0...,None,None,26,True,success,https://www.bloomberg.com/news/articles/2025-0...,,...,AI Boom Powers Triple-Digit Gains in China’s H...,1.000000,-1.323719e-08,1.000000e+00,1.000000e+00,1.480472e-01,-2.043171,0.000000,5.148047,-22.008367
69,Feedly AI,Opinion | Vivek Ramaswamy: How Workers Can Bui...,https://www.wsj.com/opinion/can-americans-buil...,None,None,69,True,success,https://www.wsj.com/opinion/can-americans-buil...,,...,Opinion | Vivek Ramaswamy: How Workers Can Bui...,1.000000,-1.323719e-08,1.000000e+00,1.192029e-01,0.000000e+00,-2.046240,0.000000,4.119203,-22.041420
138,Feedly AI,Go Experts: ‘I Don’t Want to Maintain AI-Gener...,https://thenewstack.io/go-experts-i-dont-want-...,None,None,138,True,success,https://thenewstack.io/go-experts-i-dont-want-...,download/html/Go_Experts___I_Don_t_Want_to_Mai...,...,Go Experts: ‘I Don’t Want to Maintain AI-Gener...,0.508530,4.058765e-01,0.000000e+00,9.999785e-01,1.275191e-07,-2.054671,0.954291,2.360146,-22.132239


- rate articles
- load sources into db with reputation
- get domain from each url and put in headline_df
- look up source and reputation
- prompt for on topic , important, high quality
- run bradley terry
- combine ratings


set selected flag using rating
store to db

- next steps select sections
- clean clusters , combine clusters, select sections 